In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
DATA_PATH = '../raw_data'

In [3]:
train = pd.read_csv(f'{DATA_PATH}/train.csv')
X = train.drop(columns='Yards')
y = train.Yards

/home/jeanarnaudritouret/.pyenv/versions/3.8.6/envs/kaggle_nflbowl_yards/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Categorical Features

In [30]:
train_cat = train.select_dtypes(include=['object'])

In [31]:
train_cat.nunique()

Team                     2
DisplayName           2568
GameClock              901
PossessionTeam          32
FieldPosition           32
OffenseFormation         8
OffensePersonnel        61
DefensePersonnel        45
PlayDirection            2
TimeHandoff          30709
TimeSnap             30721
PlayerHeight            16
PlayerBirthDate       1897
PlayerCollegeName      314
Position                25
HomeTeamAbbr            32
VisitorTeamAbbr         32
Stadium                 61
Location                67
StadiumType             33
Turf                    23
GameWeather             73
WindSpeed               69
WindDirection           58
dtype: int64

In [32]:
train_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682154 entries, 0 to 682153
Data columns (total 24 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Team               682154 non-null  object
 1   DisplayName        682154 non-null  object
 2   GameClock          682154 non-null  object
 3   PossessionTeam     682154 non-null  object
 4   FieldPosition      673552 non-null  object
 5   OffenseFormation   682066 non-null  object
 6   OffensePersonnel   682154 non-null  object
 7   DefensePersonnel   682154 non-null  object
 8   PlayDirection      682154 non-null  object
 9   TimeHandoff        682154 non-null  object
 10  TimeSnap           682154 non-null  object
 11  PlayerHeight       682154 non-null  object
 12  PlayerBirthDate    682154 non-null  object
 13  PlayerCollegeName  682154 non-null  object
 14  Position           682154 non-null  object
 15  HomeTeamAbbr       682154 non-null  object
 16  VisitorTeamAbbr    6

## FieldPosition

In [33]:
train_cat.FieldPosition.unique()

array(['NE', 'KC', nan, 'BUF', 'NYJ', 'ATL', 'CHI', 'CIN', 'BLT', 'CLV',
       'PIT', 'ARZ', 'DET', 'JAX', 'HST', 'TEN', 'OAK', 'WAS', 'PHI',
       'LA', 'IND', 'GB', 'SEA', 'CAR', 'SF', 'DAL', 'NYG', 'NO', 'MIN',
       'DEN', 'LAC', 'TB', 'MIA'], dtype=object)

In [34]:
train_cat.PossessionTeam.unique()

array(['NE', 'KC', 'BUF', 'NYJ', 'ATL', 'CHI', 'CIN', 'BLT', 'CLV', 'PIT',
       'ARZ', 'DET', 'JAX', 'HST', 'OAK', 'TEN', 'WAS', 'PHI', 'LA',
       'IND', 'SEA', 'GB', 'CAR', 'SF', 'DAL', 'NYG', 'NO', 'MIN', 'DEN',
       'LAC', 'TB', 'MIA'], dtype=object)

In [35]:
def possession_in_fieldPosition(df):
    return df.FieldPosition == df.PossessionTeam

## OffenseFormation

In [46]:
train_cat.OffenseFormation.value_counts()

SINGLEBACK    299728
SHOTGUN       206558
I_FORM        136950
PISTOL         21538
JUMBO          14894
WILDCAT         1694
EMPTY            682
ACE               22
Name: OffenseFormation, dtype: int64